In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from collections import defaultdict
from copy import deepcopy
import torch

torch.cuda.is_available()
import yaml
import pyhocon
import joblib
from copy import deepcopy
import json
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import simpletransformers
from sklearn.metrics import classification_report
from RobotCommandClassifier import utils
from RobotCommandClassifier import *
from MultilabelML import PrepareData, PrepareInput_for_simpletransformers, PrepareModel

In [2]:
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/SimpleLM.conf")
#CONFIG = configFileContent['rubert_tiny2_multilabel_noYno0_fold4'].as_plain_ordered_dict()
#configFileContent = pyhocon.ConfigFactory.parse_file("../Configs/CustomML.conf")
#CONFIG = configFileContent['MyMultiTiny2_data2_fold0'].as_plain_ordered_dict()
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/MyMultiTiny2/fold_0/config.json", "r") as f:
    CONFIG = json.load(f)

# если хотим протестировать на всем тесте, включая фолды, на которых он обучался
#CONFIG["Data"].pop("test_only_on_fold")

In [3]:
train_x_df, train_y_df, valid_x_df, valid_y_df, test_x_df, test_y_df = PrepareData(CONFIG)

In [4]:
CONFIG["Model"]['model_name'] = CONFIG["output_dir"] + '/models/checkpoint-63780-epoch-10'
model = PrepareModel(CONFIG)

# Make prediction

In [5]:
predictions, raw_outputs = model.predict(test_x_df.values.tolist())
#predictions = np.array(predictions)

  0%|          | 0/264 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

In [6]:
#привести бинарный мультилейбл к мультиклассовому
classes_per_attribute = CONFIG["Model"]['num_sublabels_per_biglabel']
predictions_2 = utils.binarymultilabel_to_multiclassmultilabel(raw_outputs, classes_per_attribute, CONFIG)

In [7]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
#result = utils.calculate_metrics_2(test_y_df.iloc[:,1:], predictions_2[:,1:], display=True) # исключить Y из оценки
result = utils.calculate_metrics_2(test_y_df, predictions_2, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_classes_report.json"), "w") as f:
    json.dump(result, f)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_y_df, predictions_2, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_avg_report.json"), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      0.17      0.29         6
         1.0       0.67      0.80      0.73         5
         2.0       0.80      0.89      0.84         9
         3.0       1.00      0.50      0.67         8
         5.0       0.00      0.00      0.00         0
         6.0       0.99      1.00      0.99       236
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0

    accuracy                           0.96       264
   macro avg       0.49      0.37      0.39       264
weighted avg       0.98      0.96      0.96       264

direction
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99       247
         1.0       1.00      0.25      0.40         4
         2.0       1.00      0.80      0.89         5
         3.0       0.25      1.00      0.40         1
       

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

83.0 	 correct_samples_perc
96.0 	 [action]_acc
39.0 	 [action]_macrof1
97.0 	 [direction]_acc
55.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
100.0 	 [degshours]_macrof1
97.0 	 [object1]_acc
96.0 	 [object1]_macrof1
100.0 	 [nearest]_acc
100.0 	 [nearest]_macrof1
96.0 	 [relation1]_acc
58.0 	 [relation1]_macrof1
95.0 	 [object2]_acc
95.0 	 [object2]_macrof1
96.0 	 [relation2]_acc
58.0 	 [relation2]_macrof1
97.0 	 [object3]_acc
97.0 	 [object3]_macrof1
99.0 	 [self]_acc
50.0 	 [self]_macrof1
99.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
75.0 	 avg_macro_f1
98.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [8]:
possible_combinations = pd.read_csv("../Data/Interim/possible_combinations_of_attributes_for_actions.csv")
possible_combinations_arr = possible_combinations.loc[:, CONFIG["Data"]["target_columns"]].values

In [9]:
# Rule
def softmax(x):
    #e_x = x - np.expand_dims(np.max(x, axis=1), axis=1)
    #return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)
    e_x = np.exp(x)
    return e_x / np.expand_dims(np.sum(e_x, axis=1), axis=1)
softmax_outputs = np.zeros_like(raw_outputs, dtype=np.float32)
shift = 0
for num_sublabels in CONFIG['Model']['num_sublabels_per_biglabel']:
    softmax_outputs[:, shift:shift + num_sublabels] = softmax(raw_outputs[:, shift:shift + num_sublabels])
    shift += num_sublabels

predictions_3 = []
for i in range(len(softmax_outputs)):
    shift = 0
    probs_for_combinations = np.zeros_like(possible_combinations_arr, dtype=np.float32)
    # предполагается, что первые значения в выходном векторе из модели - классы экшена
    assert CONFIG["Data"]["target_columns"][0]=="action"
    probs_for_combinations[:,0] = np.take(softmax_outputs[i, shift:shift+classes_per_attribute[0]], possible_combinations_arr[:,0])
    shift += classes_per_attribute[0]
    #print(probs_for_combinations[:,0])
    maxprob_attribute_classes = [-1] # -1 for action
    for attribute_i in range(1, len(classes_per_attribute)):
        # есть вариант ставить 0 для нулевых классов или наоборот - обратное от максимального класса
        probs = softmax_outputs[i, shift:shift+classes_per_attribute[attribute_i]]
        assert np.round(sum(probs), 5)==1
        zerocls_prob = probs[0]
        nonzerocls_max_prob = np.max(probs[1:])
        #print(zerocls_prob, nonzerocls_max_prob)
        maxprob_attribute_classes.append(np.argmax(probs[1:])+1)
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==0, attribute_i] = zerocls_prob
        probs_for_combinations[possible_combinations_arr[:, attribute_i]==1, attribute_i] = nonzerocls_max_prob
        shift += classes_per_attribute[attribute_i]

    best_template_i = np.argmax(np.sum(probs_for_combinations, axis=1))
    sample_prediction = possible_combinations_arr[best_template_i].copy()
    for i in range(1, len(classes_per_attribute)):
        if sample_prediction[i]!=0:
            sample_prediction[i]=maxprob_attribute_classes[i]
    predictions_3.append(sample_prediction)
predictions_3 = np.array(predictions_3)

In [10]:
if not os.path.exists(os.path.join(CONFIG["output_dir"], "reports")):
    os.mkdir(os.path.join(CONFIG["output_dir"], "reports"))
result = utils.calculate_metrics_2(test_y_df, predictions_3, display=True)
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_classes_report_rule.json"), "w") as f:
    json.dump(result, f)

result_avg = utils.calculate_metrics(test_y_df, predictions_3, config={
    "report_metrics": CONFIG["Report"]["report_metrics"]
})
with open(os.path.join(CONFIG["output_dir"], "reports/epoch-10_avg_report_rule.json"), "w") as f:
    json.dump(result_avg, f)
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

           0       1.00      0.17      0.29         6
           1       0.50      0.80      0.62         5
           2       0.75      0.67      0.71         9
           3       1.00      0.50      0.67         8
           5       0.00      0.00      0.00         0
           6       0.99      1.00      0.99       236
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.95       264
   macro avg       0.47      0.35      0.36       264
weighted avg       0.97      0.95      0.95       264

direction
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       247
           1       1.00      0.25      0.40         4
           2       1.00      0.80      0.89         5
           3       0.25      1.00      0.40         1
       

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

84.0 	 correct_samples_perc
95.0 	 [action]_acc
36.0 	 [action]_macrof1
97.0 	 [direction]_acc
55.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
96.0 	 [object1]_acc
84.0 	 [object1]_macrof1
100.0 	 [nearest]_acc
100.0 	 [nearest]_macrof1
96.0 	 [relation1]_acc
58.0 	 [relation1]_macrof1
95.0 	 [object2]_acc
95.0 	 [object2]_macrof1
96.0 	 [relation2]_acc
58.0 	 [relation2]_macrof1
97.0 	 [object3]_acc
97.0 	 [object3]_macrof1
99.0 	 [self]_acc
50.0 	 [self]_macrof1
99.0 	 [gaze]_acc
50.0 	 [gaze]_macrof1
69.0 	 avg_macro_f1
98.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu